## Final Tutorial outline

1. Define unmitigated executor
2. compute unmitigated expval
3. mitigate with REM
4. mitigate with ZNE
5. mitigate with REM + ZNE
6. compare results

In [75]:
# step 1
import cirq
import numpy as np
from mitiq.benchmarks import generate_rb_circuits
from mitiq import MeasurementResult, Observable, PauliString, raw

circuit = generate_rb_circuits(2, 5)[0]

p0 = 0.05
p1 = p0

def execute(circuit: cirq.Circuit, noise_level=0.005) -> MeasurementResult:
    measurements = circuit[-1]
    circuit =  circuit[:-1]
    circuit = circuit.with_noise(cirq.depolarize(noise_level))
    circuit.append(cirq.bit_flip(p0).on_each(circuit.all_qubits()))
    circuit.append(measurements)

    simulator = cirq.DensityMatrixSimulator()

    result = simulator.run(circuit, repetitions=10000)
    bitstrings = np.column_stack(list(result.measurements.values()))
    return MeasurementResult(bitstrings)


# step 2
obs = Observable(PauliString("ZI"), PauliString("IZ"))
result = raw.execute(circuit, execute, obs)
print(result)

(1.244+0j)


In [76]:
# step 3 (REM)
from mitiq import rem

icm = rem.generate_inverse_confusion_matrix(2, p0, p1)
rem_executor = rem.mitigate_executor(execute, inverse_confusion_matrix=icm)

obs.expectation(circuit, rem_executor)

(1.3766+0j)

In [77]:
# step 4 (ZNE)
from mitiq import zne

zne_executor = zne.mitigate_executor(execute, observable=obs)

zne_executor(circuit)

(1.7842+0j)

In [78]:
# step 5 (REM + ZNE)

both_executor = zne.mitigate_executor(rem_executor, observable=obs)

both_executor(circuit)

(1.9557999999999998+0j)